In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
from tensorflow.python.ops import rnn, rnn_cell

# Hyperparameters

In [2]:
sec_per_ar = 0.5
ar_per_sec = 1/sec_per_ar
time_length = 1000
train_steps = 10000
check_steps = 1000

# Load patient ar_data

In [ ]:
folder_path = '/home/maestoj/medical_analysis/ar100/'

In [ ]:
def ar_data_load(patient_number,data_set_number):
    file_name = "data_patient_{}_AR{}.csv".format(patient_number,data_set_number)
    file_path = folder_path+file_name
    ar_file = open(file_path,'r',newline='')
    reader = csv.reader(ar_file, delimiter= ',')
    temp = []
    for row in reader:
        temp.append(row)
    temp = np.array(temp, dtype = np.float32)
    print("{} is loaded from {}".format(file_name, file_path))
    return temp

In [ ]:
def ar_data_average_channels(ar_datas):
    a, b = ar_datas.shape
    temp = np.zeros(b)
    for i in range(b):
        for j in range(a):
            temp[i] += ar_datas[j][i]
        temp[i] /=a
    return temp

In [ ]:
def ar_data_select_channel(ar_datas, channel_number):
    return ar_datas[channel_number - 1]

In [ ]:
def normalizer(array):
    temp_max = np.max(array)
    temp_min = np.min(array)
    temp = np.zeros(len(array))
    for i in range(len(array)):
        temp[i] = (array[i]-temp_min)/(temp_max-temp_min)
    return temp

In [ ]:
def time_axis_maker(array, sec_per_cell,init = 0):
    final = init + sec_per_cell*(len(array)-1)
    return np.linspace(init,final,len(array))

# Test functions and librosa test

In [ ]:
y = ar_data_average_channels(ar_data_load(1,1))
time = time_axis_maker(y,sec_per_ar)
plt.plot(time, y)
plt.show()

# percurssion graph
for patient_number in range(1, len(onset_times)+1):
    for dataset_number in range(1, event_number[patient_number-1]+1):
        y = ar_data_sum_channels(ar_data_load(patient_number,dataset_number))
        temp = time_axis_maker(y,sec_per_ar)
        temp_max = np.max(y)
        plt.plot(temp, y)
        plt.title('{}, {}'.format(patient_number, dataset_number))
        plt.vlines(onset_times[patient_number-1][dataset_number-1],0,temp_max, color='r', alpha=0.9,
                        linestyle='--', label='Onsets')
        plt.show()
        y_harm, y_perc = librosa.effects.hpss(y)
        librosa.display.waveplot(y_perc, alpha=0.5)
        plt.show()

# librosa stft
D_short = librosa.stft(y,win_length = 100, hop_length = 100)
print(D_short.shape)
librosa.display.specshow(abs(D_short))
plt.show()

# Labeling read

In [ ]:
label_file = open('model_v1_label.csv','r',newline='')
reader = csv.reader(label_file,delimiter=',')
label = []
for row in reader:
    label.append(row)
label = np.array(label)
label = label.astype(np.float)
print(label)

# Onset-time data

In [ ]:
def seizure_time_parser(onset_times,patient_number,data_set_number):
    temp = str(onset_times[patient_number-1][data_set_number])
    curr_onset_time = []
    while temp.find('/')!=-1:
        curr_onset_time.append(int(temp[0:temp.find('/')]))
        temp=temp[temp.find('/')+1:]
    curr_onset_time.append(int(float(temp)))
    curr_onset_time=np.array(curr_onset_time)
    return curr_onset_time

In [ ]:
seizure_file = open('seizure_times.csv','r',newline='')
reader = csv.reader(seizure_file,delimiter=',')
onset_times_temp=[]

for row in reader:
    onset_times_temp.append(row)
    
onset_times = []
total_patient = len(onset_times_temp)
event_number = np.zeros(total_patient,dtype=np.int32)
#number of events of (i+1) patient = event_number[i]

for i in range(total_patient):
    event_number[i] = len(onset_times_temp[i])-1

for p in range(total_patient):
    temp = []
    for d in range(event_number[p]):
        temp.append(seizure_time_parser(onset_times_temp,p+1,d+1))
    onset_times.append(temp)

In [ ]:
print("The number of datasets of patients")
print(event_number)
print("2st patient's seizure time for each data_set")
print(onset_times[1])
print("9th patient's seizure time for each data_set")
print(onset_times[8])

# Training_set and test_set generator

In [ ]:
print(len(label))
i=51
patient_number = int(label[i][0])
dataset_number = int(label[i][1])
temp_output = label[i][2]
print(patient_number, dataset_number)
temp = normalizer(ar_data_average_channels(ar_data_load(patient_number, dataset_number)))
time = time_axis_maker(temp,sec_per_ar)
plt.plot(time, temp)
plt.show()

print(len(temp))
onset_time_temp = onset_times[patient_number-1][dataset_number-1]            
print(onset_time_temp)
temp_input_data = np.zeros(time_length)

onset_on_ar = int(onset_time_temp*ar_per_sec)
print(onset_on_ar)
for i in range(time_length):
    temp_input_data[i] = temp[onset_on_ar-time_length+1+i]
time2 = time_axis_maker(temp_input_data,sec_per_ar,(onset_on_ar-time_length)*sec_per_ar)

y_max = np.max(temp_input_data)
plt.plot(time2,temp_input_data)
plt.vlines(onset_time_temp-temp_output, 0, y_max, colors='r', linestyles='--')
plt.show()

print(temp_output)

train_input = []
train_output = []
test_input = []
test_output = []


In [ ]:
def train_and_test_set_generator():
    
    train_input = []
    train_output = []
    test_input = []
    test_output = []
    total_data_number = len(label)
    train_data_number = total_data_number*0.8
    for label_seq in range(total_data_number):
        patient_number = int(label[label_seq][0])
        dataset_number = int(label[label_seq][1])

        temp = normalizer(ar_data_average_channels(ar_data_load(patient_number, dataset_number)))
        onset_time_temp = onset_times[patient_number-1][dataset_number-1]            
        
        if len(onset_time_temp) > 1:
            print("More than one onset time")
            continue
    
        onset_time_temp = onset_time_temp[0]
    
        if onset_time_temp*ar_per_sec < time_length:
            print("Wrong input")
            continue
        
        temp_input_data = np.zeros(time_length)
        temp_output_data = label[label_seq][2]
        
        onset_on_ar = int(onset_time_temp*ar_per_sec)
        
        if onset_on_ar > len(temp):
            print("Out of boundary")
            continue
        
        for i in range(time_length):
            temp_input_data[i] = temp[onset_on_ar-time_length+1+i]

        if label_seq < train_data_number:
            train_input.append(temp_input_data)
            train_output.append(temp_output_data)
        else:
            test_input.append(temp_input_data)
            test_output.append(temp_output_data)
            
    train_input = np.array(train_input)
    train_output = np.array(train_output)
    test_input = np.array(test_input)
    test_output = np.array(test_output)
    return {'train_input' : train_input, 'train_output' : train_output, 
            'test_input' : test_input,'test_output' : test_output}

In [ ]:
result = train_and_test_set_generator()
train_input = result['train_input']
train_output = result['train_output']
test_input = result['test_input']
test_output = result['test_output']
train_output = np.reshape(train_output,[-1,1])
test_output = np.reshape(test_output,[-1,1])
print(train_input.shape)
print(train_output.shape)
print(test_input.shape)
print(test_output.shape)

# Neural network

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, time_length])
y = tf.placeholder(tf.float32, shape=[None, 1])

weights = {"w_conv1" : tf.Variable(tf.truncated_normal([5, 1, 1, 16], stddev=0.1)),
           "w_conv2" : tf.Variable(tf.truncated_normal([5, 1, 16, 32], stddev=0.1)),
           "w_fc1" : tf.Variable(tf.truncated_normal([40*32, 100], stddev=0.1)),
           "w_out" : tf.Variable(tf.truncated_normal([100, 1], stddev=0.1))
          }

biases = {"b_conv1" : tf.Variable(tf.constant(0.1, shape=[16])),
          "b_conv2" : tf.Variable(tf.constant(0.1, shape=[32])),
            "b_fc1" : tf.Variable(tf.constant(0.1, shape=[100])),
           "b_out" : tf.Variable(tf.constant(0.1, shape=[1]))
        }
def conv1d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool1d(x):
    return tf.nn.max_pool(x, ksize=[1,5,1,1], strides=[1,5,1,1], padding='SAME')

train_x = tf.reshape(x, [-1,time_length,1,1])
conv1 = conv1d(train_x,weights["w_conv1"])
relu1 = tf.nn.relu(conv1 + biases["b_conv1"])
pool1 = maxpool1d(relu1)
print(relu1)
print(pool1)

conv2 = conv1d(pool1,weights["w_conv2"])
relu2 = tf.nn.relu(conv2 + biases["b_conv2"])
pool2 = maxpool1d(relu2)
print(relu2)
print(pool2)

pool2_temp = tf.reshape(pool2, [-1,40*32])
h1 = tf.nn.relu(tf.matmul(pool2_temp, weights["w_fc1"])+ biases["b_fc1"])
y_hat = tf.matmul(h1, weights["w_out"])+ biases["b_out"]
print(y_hat)
#h_conv = tf.nn.conv2d(x_image, W_conv, strides=[1, 1, 1, 1], padding='VALID')
#h_relu = tf.nn.relu(h_conv + b_conv)
#h_pool = tf.nn.max_pool(h_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



In [ ]:
cost = tf.reduce_mean(tf.square(y_hat-y))
train = tf.train.AdamOptimizer(1e-3).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
plt.plot(train_output)
plt.show()
for i in range(train_steps):
    opt, c = sess.run([train, cost], feed_dict={x: train_input, y: train_output})
    if i%int(check_steps)==0:
        print("Cost = {}".format(c))
        plt.plot(sess.run(y_hat, feed_dict = {x : train_input}))
        plt.show()

In [ ]:
print(sess.run(cost, feed_dict={x: test_input,y:test_output}))
plt.plot(test_output)
plt.show()
plt.plot(sess.run(y_hat, feed_dict = {x : test_input}))
plt.show()